*Installing necessary packages for the proper functioning of the model

In [1]:
!pip install Keras #installing Keras

In [2]:
!pip install --upgrade pip #Upgrading pip 


Requirement already up-to-date: pip in /Users/oindrilamukherjee/opt/anaconda3/lib/python3.7/site-packages (20.2.4)


In [3]:
!pip install tensorflow #installing Tensorflow

In [7]:
#Importing necessary packages for the functioning 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras import layers
from keras.models import Model
from keras import Input
from keras.layers import LSTM
from keras.utils import to_categorical

In [8]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [9]:
# Copying the datapath from the source
train_data_file = '/kaggle/input/fi2010/FI2010/FI2010_train.csv'
test_data_file = '/kaggle/input/fi2010/FI2010/FI2010_test.csv'

* Just in case if this code is processed in google colab the way to upload the datset is
*loading dataset
from google.colab import files
uploaded = files.upload()*

In [13]:
#Reading the dataset
#Importing the csv file 
#Training set and Testing set separately
tr_d = pd.read_csv('FI2010_train.csv', index_col=0)
print(tr_d)

               0         1         2         3         4         5         6  \
0       0.318116 -0.564619  0.313539 -0.551889  0.319726 -0.731228  0.312891   
1       0.318116 -0.662079  0.313539 -0.551889  0.320706 -0.751891  0.312891   
2       0.317136 -0.723163  0.313539 -0.551889  0.316787 -0.731228  0.312891   
3       0.317136 -0.585895  0.313539 -0.551889  0.318747 -0.307628  0.312891   
4       0.317136 -0.585895  0.313539 -0.551889  0.318747 -0.307628  0.312891   
...          ...       ...       ...       ...       ...       ...       ...   
362395  1.328480 -0.608544  1.329517 -0.550603  1.327892 -0.795514  1.330085   
362396  1.328480 -0.608544  1.329517 -0.550603  1.327892 -0.795514  1.330085   
362397  1.328480 -0.608544  1.329517 -0.550603  1.327892 -0.795514  1.330085   
362398  1.328480 -0.668256  1.329517 -0.354503  1.327892 -0.795514  1.330085   
362399  1.328480 -0.668256  1.329517 -0.354503  1.327892 -0.795514  1.330085   

               7         8         9  .

In [15]:
tr_d.shape# dimension the training dataset

(362400, 149)

In [16]:
#uploading test data
te_d = pd.read_csv('FI2010_test.csv', index_col=0)
print(te_d)

              0         1         2         3         4         5         6  \
0      0.381815 -0.669628  0.380290 -0.453518  0.383410 -0.454568  0.380638   
1      0.381815 -0.669628  0.382253 -0.567963  0.383410 -0.454568  0.380638   
2      0.381815 -0.669628  0.379308 -0.440016  0.382431 -0.788626  0.375729   
3      0.381815 -0.669628  0.380290 -0.567963  0.382431 -0.788626  0.380638   
4      0.381815 -0.669628  0.378326 -0.549317  0.382431 -0.788626  0.378674   
...         ...       ...       ...       ...       ...       ...       ...   
31932  1.462738 -0.030648  1.467926 -0.197623  1.462118 -0.401187  1.465580   
31933  1.462738 -0.030648  1.467926 -0.197623  1.462118 -0.401187  1.465580   
31934  1.462738 -0.030648  1.467926 -0.197623  1.462118 -0.401187  1.465580   
31935  1.462738 -0.030648  1.467926 -0.197623  1.462118 -0.401187  1.466562   
31936  1.462738 -0.030648  1.467926 -0.197623  1.462118 -0.401187  1.465580   

              7         8         9  ...       139 

In [17]:
te_d.shape#dimension of test data

(31937, 149)

*Data Preprocessing :technique that involves transforming raw data into an understandable format. Real-world data is often incomplete, inconsistent, and/or lacking in certain behaviors or trends, and is likely to contain many errors. Data preprocessing is a proven method of resolving such issues.
*Training of the dataset 

In [18]:
# ready data for training:
# 1. sample_size=100: the most 100 recent updates
# 2. feature_num=40: 40 features per time stamp
# 3. target_num=5: relative changes for the next 1,2,3,5 and 10 events(5 in total)
def data_generator(data, batch_size=32, lookback=100,
                   feature_num=40, target_delay=1, shuffle=False):
    data = data.values
    shape = data.shape
    max_index = shape[0]
    min_index = 0
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(min_index + lookack, max_index)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows), lookback, feature_num))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            samples[j] = data[row - lookback: row, 0: feature_num]  # take the first 40 columns as features
            targets[j] = data[row - 1, target_delay - 6]
        samples = samples.reshape(samples.shape[0], samples.shape[1],
                                  samples.shape[2], 1)# add the 4th dimension: 1 channel
        # "Benchmark dataset for mid-price forecasting of limit order book data with machine learning"
        # labels 1: equal to or greater than 0.002
        # labels 2: -0.00199 to 0.00199
        # labels 3: smaller or equal to -0.002
        # Y=Y-1 relabels as 0,1,2
        targets = targets - 1
        targets = targets.astype(int)
        targets = to_categorical(targets, num_classes=3)# y is the next event's mid price (k=1)
        yield samples, targets

In [19]:
# the size of a single input is (100,40)
input_tensor = Input(shape=(100,40,1))

# convolutional filter is (1,2) with stride of (1,2)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

# Inception Module
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

# concatenate features of tower_1, tower_2, tower_3
layer_x = layers.Reshape((100,96))(layer_x)

# 64 LSTM units
layer_x = LSTM(64)(layer_x)
# The last output layer uses a softmax activation function
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 40, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 20, 16)  48          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 100, 20, 16)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 20, 16)  1040        leaky_re_lu[0][0]                
_______________________________________________________________________________________

In [23]:
batch_size = 32
steps_per_epoch =  len(train_data) // batch_size
train_gen = data_generator(train_data, batch_size=batch_size, lookback=100,
                   feature_num=40, target_delay=1, shuffle=False)
opt = keras.optimizers.RMSprop(lr=0.01, epsilon=1)# learning rate and epsilon are the same as paper DeepLOB
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit_generator(train_gen,
                    steps_per_epoch=steps_per_epoch,
                    epochs=50)

Epoch 1/50
11325/11325 [==============================] - 9926s 876ms/step - loss: 0.8804 - accuracy: 0.6391
Epoch 2/50
11325/11325 [==============================] - 2157s 191ms/step - loss: 0.8156 - accuracy: 0.6467
Epoch 3/50
11325/11325 [==============================] - 2294s 203ms/step - loss: 0.6769 - accuracy: 0.7013
Epoch 4/50
11325/11325 [==============================] - 31447s 3s/step - loss: 0.6588 - accuracy: 0.7112
Epoch 5/50
11325/11325 [==============================] - 2773s 245ms/step - loss: 0.6164 - accuracy: 0.7541
Epoch 6/50
11325/11325 [==============================] - 2771s 245ms/step - loss: 0.5803 - accuracy: 0.7744
Epoch 7/50
11325/11325 [==============================] - 2608s 230ms/step - loss: 0.5722 - accuracy: 0.7781
Epoch 8/50
11325/11325 [==============================] - 2333s 206ms/step - loss: 0.5668 - accuracy: 0.7807
Epoch 9/50
11325/11325 [==============================] - 2234s 197ms/step - loss: 0.5614 - accuracy: 0.7828
Epoch 10/50
11325/113